In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image

# Directorio donde están las imágenes organizadas por carpetas (una por cada set)
data_dir = "../TRABAJO/FOTOS"

# Transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar todas las imágenes
    transforms.RandomHorizontalFlip(),  # Aumento de datos: reflejo horizontal
    transforms.RandomRotation(15),  # Aumento de datos: rotación aleatoria
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Ajuste de color
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize([0.5], [0.5])  # Normalizar valores de píxeles
])

# Cargar dataset con ImageFolder (cada carpeta es una clase)
dataset = ImageFolder(root=data_dir, transform=transform)

# Dividir en entrenamiento (80%), validación (10%) y prueba (10%)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Crear DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Dataset preprocesado y dividido:")
print(f"- Entrenamiento: {len(train_dataset)} imágenes")
print(f"- Validación: {len(val_dataset)} imágenes")
print(f"- Prueba: {len(test_dataset)} imágenes")


Dataset preprocesado y dividido:
- Entrenamiento: 1062 imágenes
- Validación: 132 imágenes
- Prueba: 134 imágenes


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Comprobar si hay GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo preentrenado EfficientNetB0
model = models.efficientnet_b0(pretrained=True)

# Modificar la última capa para clasificar nuestros sets de LEGO
num_features = model.classifier[1].in_features
num_classes = len(dataset.classes)  # Número de sets de LEGO
model.classifier[1] = nn.Linear(num_features, num_classes)

# Enviar el modelo a GPU si está disponible
model = model.to(device)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_acc = 100 * correct / total
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    # Evaluación en conjunto de validación
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_acc = 100 * correct_val / total_val

    print(f"Época {epoch+1}/{num_epochs} - Pérdida: {running_loss/len(train_loader):.4f} - "
          f"Acc. Entrenamiento: {train_acc:.2f}% - Acc. Validación: {val_acc:.2f}%")

# Guardar el modelo entrenado
torch.save(model.state_dict(), "../04_Extra/ID/modelo_lego.pth")
print("Entrenamiento finalizado. Modelo guardado.")


/Users/luismgl/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/luismgl/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /Users/luismgl/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:32<00:00, 669kB/s] 


Época 1/20 - Pérdida: 3.0010 - Acc. Entrenamiento: 29.94% - Acc. Validación: 61.36%
Época 2/20 - Pérdida: 1.1246 - Acc. Entrenamiento: 73.35% - Acc. Validación: 69.70%
Época 3/20 - Pérdida: 0.5109 - Acc. Entrenamiento: 87.76% - Acc. Validación: 80.30%
Época 4/20 - Pérdida: 0.3732 - Acc. Entrenamiento: 90.77% - Acc. Validación: 76.52%
Época 5/20 - Pérdida: 0.2413 - Acc. Entrenamiento: 94.54% - Acc. Validación: 81.06%
Época 6/20 - Pérdida: 0.2301 - Acc. Entrenamiento: 94.16% - Acc. Validación: 81.06%
Época 7/20 - Pérdida: 0.1565 - Acc. Entrenamiento: 96.05% - Acc. Validación: 81.06%
Época 8/20 - Pérdida: 0.0919 - Acc. Entrenamiento: 98.31% - Acc. Validación: 81.06%
Época 9/20 - Pérdida: 0.0751 - Acc. Entrenamiento: 98.40% - Acc. Validación: 84.85%
Época 10/20 - Pérdida: 0.0764 - Acc. Entrenamiento: 98.21% - Acc. Validación: 82.58%
Época 11/20 - Pérdida: 0.1471 - Acc. Entrenamiento: 96.42% - Acc. Validación: 78.79%
Época 12/20 - Pérdida: 0.1310 - Acc. Entrenamiento: 97.27% - Acc. Validaci

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import numpy as np
import json
from collections import Counter

# Probamos si tenemos GPU disponible para el procesamiento
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Definimos el directorio del dataset de fotos de LEGO
data_dir = "../TRABAJO/FOTOS"

# Configuramos Data Augmentation optimizado para imágenes 
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Cargamos el dataset
dataset = ImageFolder(root=data_dir, transform=transform)
num_classes = len(dataset.classes)

# Guardamos el mapeo de clases a nombres de sets
class_to_idx = dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}
with open("idx_to_class.json", "w") as f:
    json.dump(idx_to_class, f)

# Dividimos el dataset
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Definimos el balanceo de clases con pesos
train_targets = [dataset.targets[i] for i in train_dataset.indices]
class_counts = Counter(train_targets)
class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
samples_weights = [class_weights[t] for t in train_targets]
sampler = WeightedRandomSampler(samples_weights, num_samples=len(samples_weights), replacement=True)

# Optimizamos un DataLoader para cargar datos más rápido en entrenamiento y validación
batch_size = 32
num_workers = 4 
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Modelo EfficientNet-B0 
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
model = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
num_features = model.classifier[1].in_features

# Modificamos la última capa para que se ajuste a nuestro número de clases a los números de los sets de LEGO
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Linear(512, num_classes)
)

model = model.to(device)

# Configuramos el entrenamiento
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # Adam en lugar de SGD (más eficiente)
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)
criterion = nn.CrossEntropyLoss()

# Definimos el Early Stopping para detener el entrenamiento si no mejora la validación en 2 épocas
early_stopping_patience = 2 
best_val_loss = np.inf
epochs_no_improve = 0

# Definimos la función de validación para evaluar el modelo en el conjunto de validación
def validate(model, val_loader, criterion):
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    avg_val_loss = running_val_loss / len(val_loader)
    val_acc = 100 * correct_val / total_val
    return avg_val_loss, val_acc

# Definimos el entrenamiento con validación y Early Stopping anteriores
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        running_loss += loss.item()

    train_acc = 100 * correct_train / total_train
    avg_train_loss = running_loss / len(train_loader)

    # Validación
    avg_val_loss, val_acc = validate(model, val_loader, criterion)

    print(f"Época {epoch+1}/{num_epochs} - "
          f"Pérdida Entrenamiento: {avg_train_loss:.4f} - Acc. Entrenamiento: {train_acc:.2f}% | "
          f"Pérdida Validación: {avg_val_loss:.4f} - Acc. Validación: {val_acc:.2f}%")

    # Early Stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), "modelo_lego_final.pth")
        print("✅ Modelo mejorado y guardado.")
    else:
        epochs_no_improve += 1
        print(f"⚠️ No hay mejora en validación por {epochs_no_improve} épocas.")

    if epochs_no_improve >= early_stopping_patience:
        print("🛑 Detención temprana activada. Entrenamiento finalizado.")
        break

print("🎉 Entrenamiento finalizado. Modelo guardado como 'modelo_lego_final.pth'.")


Usando dispositivo: cpu
Época 1/30 - Pérdida Entrenamiento: 2.9137 - Acc. Entrenamiento: 35.50% | Pérdida Validación: 1.4996 - Acc. Validación: 52.27%
✅ Modelo mejorado y guardado.
Época 2/30 - Pérdida Entrenamiento: 1.0102 - Acc. Entrenamiento: 73.63% | Pérdida Validación: 0.8085 - Acc. Validación: 75.00%
✅ Modelo mejorado y guardado.
Época 3/30 - Pérdida Entrenamiento: 0.6643 - Acc. Entrenamiento: 80.51% | Pérdida Validación: 0.7736 - Acc. Validación: 73.48%
✅ Modelo mejorado y guardado.
Época 4/30 - Pérdida Entrenamiento: 0.4371 - Acc. Entrenamiento: 88.51% | Pérdida Validación: 0.6874 - Acc. Validación: 77.27%
✅ Modelo mejorado y guardado.
Época 5/30 - Pérdida Entrenamiento: 0.3571 - Acc. Entrenamiento: 90.30% | Pérdida Validación: 0.7198 - Acc. Validación: 72.73%
⚠️ No hay mejora en validación por 1 épocas.
Época 6/30 - Pérdida Entrenamiento: 0.3962 - Acc. Entrenamiento: 89.92% | Pérdida Validación: 0.5314 - Acc. Validación: 82.58%
✅ Modelo mejorado y guardado.
Época 7/30 - Pérdid

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Cargamos el modelo entrenado
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("modelo_lego_final.pth", map_location=device))
model.eval()

# Definimos las transformaciones de la imagen nueva
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Cargamos imagen de prueba
image_path = "../04_Extra/ID/IMAGEN3.jpg"
image = Image.open(image_path)
image = transform(image).unsqueeze(0).to(device)

# Intentamos la predicción
with torch.no_grad():
    output = model(image)
    predicted_class = output.argmax(1).item()

# Cargamos mapeo de clases a nombres de sets
import json
with open("idx_to_class.json", "r") as f:
    idx_to_class = json.load(f)

print(f"🟢 Predicción: {idx_to_class[str(predicted_class)]}")


🟢 Predicción: 71748


/var/folders/41/9xs275yn7jxbs_h59j3pq3sw0000gn/T/ipykernel_64123/2836583553.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("modelo_lego